# `PMOIRED` Tutorial #1
## How to fit stellar diameters: uniform, limb-darkened, oblate, spotted, etc.

We will analyse the CHARA/MIRC-X data of CL Lac / IRC+50448, a Mira type AGB star. The data, which have been published in [Chiavassa et al. 2020](https://ui.adsabs.harvard.edu/abs/2020A%26A...640A..23C/abstract), and are available on [OIdB](https://oidb.jmmc.fr/search.html?conesearch=IRC%2B50448%2CJ2000%2C2%2Carcmin&perpage=50&instrument=MIRC&cs_radius_unit=arcmin&cs_equinox=J2000&order=t_min&caliblevel=3&category=SCIENCE&cs_radius=2&cs_position=IRC%2B50448). A subset of the data are provided with the tutorial.

In this tutorial, we will see how to:
- [load the data and display OIFITS data](#load)
- [fit a uniform disk model](#uniform_disk)
- [fit a limb-darkenend disk model](#limb-darkened_disk)
- [add a fully resolved component and oblatness to the star shape](#resolved)
- [add a spot on the star surface](#spot) 
- [use grid search to find the overall best position for the spot](#grid_search)
- [Bonus: use bootstrapping to evaluate uncertainties](#bootstrapping)
- [Bonus: try a dark spot rather than a bright one](#darkspot)

PMOIRED available at https://github.com/amerand/PMOIRED - tutorial by amerand@eso.org, Feb 2023

In [ ]:
%matplotlib widget
import os, sys

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
try:
    import pmoired
    print('global installation')
except:
    sys.path = ['../pmoired'] + sys.path
    import __init__ as pmoired
    print('local installation')

# -- in case you want to limit the number of cores for the multiprocessing functions
# -- will take all CPUs otherwise! 
# pmoired.MAX_THREADS = 8 
print('will use', pmoired.MAX_THREADS, 'CPUs for multiprocessing')
    
allfits = {} # we will keep track of oaa the different models we fit to the data

# Load and preview files <a id='load'></a>

OIFITS data are in `./CL_Lac` (files names end in 'viscal.fits'). Use `oi = pmoired.OI(...)` to load the files and construct your object `oi`. the function `OI()` takes simply a list of file names. `oi.show()` will show the all the data. interesting options: 
- `logV=True` to show visibilities (amplitude or squared) in log scale.
- `showFlagged=True` to show flagged data (i.e. not taken into account).
- other options are described in the doc: `?pmoired.OI`

In [ ]:
directory = './CL_Lac/'
files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('viscal.fits')]
display(files)
oi = pmoired.OI(files)
oi.show(logV=True) # possible with 'logV=True' to see low visibilities

# Uniform disk fit <a id='unfiform_disk'></a>

Using `oi.setupFit`, define the context of the fit using a dictionnary of options (check `?oi.setupFit`):
- declare that we fit the `V2` data: `'obs':['V2']`  
- We use `'max relative error':{'V2':0.5}` to ignore the squared visibilities with relative uncertainites ($\sigma_{V^2}/V^2$) larger than 50%. 
- check `?oi.setupFit` to check all options

Then we use `oi.doFit` to fit the data. `oi.doFit` takes as fist argument a dictionnary describing a model. By default, all parameters will be fitted. 

To see how write models as dictionnaries in `PMOIRED`, please refer to [the model definition notebook](https://github.com/amerand/PMOIRED/blob/master/examples/Model%20definitions%20and%20examples.ipynb). Let's start with a uniform disk of diameter 1.0 mas: `{'ud':1.0}`. Use different values as initial guess. It is better to start with a diameter too small than too large: try an initial guess >20mas. 

_As a general rule (for any parameters): do not start with exactly 0. The gradient descent algorithm has trouble choosing a step._ 

**LIMITATIONS**: the models will always disagree in the nulls ($B/\lambda\sim$100 and 200): this is because `PMOIRED` does not take into account  (yet) bandwidth smearing, which is when the visibility varies substantially within a spectral channel.

The result of the best fit is shown, and it can also be accessed in the dictionnary `oi.bestfit`. The basic informations are:
- `oi.bestfit['best']` contains the best model
- `oi.bestfit['uncer']` contains the uncertainties
- `oi.bestfit['chi2']` contains the final reduced $\chi^2$

In [ ]:
oi.setupFit({'obs':['V2'],'max relative error':{'V2':0.5}})
oi.doFit({'ud':2}) # try several first values
allfits['UD'] = oi.bestfit.copy() # saving fit for later comparison
oi.show(logV=True)

# Limb darkened disk <a id='limb-dakened_disk'></a>
## using Claret (2000) 4-parameters

from [Claret (2000)](https://ui.adsabs.harvard.edu/abs/2000A%26A...363.1081C), table [J/A+A/363/1081/atlas](https://vizier.cds.unistra.fr/viz-bin/VizieR-3?-source=J/A%2bA/363/1081/atlas&-out.max=50&-out.form=HTML%20Table&-out.add=_r&-out.add=_RAJ,_DEJ&-sort=_r&-oc.form=sexa) get the 4-coef CLD parameters: $I(\mu)/I(1) = 1-\sum_{k=1}^{4}a_k(1-\mu^{k/2})$. In this context, $\mu = \cos(\gamma) = \sqrt{1-r^2}$, $\gamma$ being the angle between the line of sight and the emergent intensity and $r$ the normalised radial distance from the centre of the star to its limb. 

Based on the stellar parameters in [Chiavassa et al. 2020](https://ui.adsabs.harvard.edu/abs/2020A%26A...640A..23C/abstract), you can use Teff=3500K and logg=1.0 (we can take VT=2.0km/s and logM/H=0.0) in the H band for MIRCX which gives $a_1$=0.7708, $a_2$=-0.05, $a_3$=-0.1577, $a_4$=0.0481

In `PMOIRED`, you describe a disk with arbitrary profile using `diam` and `profile`. The diameter is in milliarcseconds, and the profile is a string using special names `$R` and `$MU` and any additional parameters you need: for example, a linear limb-darkened disk, parametrised with `u`, will be entered in `PMOIRED` as `{'diam':2.0, 'profile':'1-$u*(1-$MU)', 'u':0.1}`. Note that refering to `u` requires a `$` in profile.

We should first fix the LD parameters (the $a_k$) using the option `doNotFit=[...]` in `oi.doFit` to list the parameters we do not want to fit. 

In a second fit, we try to fit them: the fit does not converge (see thet message and the fact that uncertainties are not computed). You can inspect the fit with `oi.showfit()` which shows the evolution of the parameters during the fitting: use the mouse to zoom and inspect the convergence.

in `oi.show()`, we can display a synthetic image by giving a field-of-view parameter (in mas): `imFov=3` If you know the parallax to the object, you can add `imPlx=` (in mas) to get a secondary scale. 

In [ ]:
oi.setupFit({'obs':['V2'], 'max relative error':{'V2':0.5}})
# -- Teff=3500, logg=1.0, fixed parameters
oi.doFit({'diam':2.5, 
          'profile':'1 - $A1*(1-$MU**(1/2)) - $A2*(1-$MU**(2/2)) - $A3*(1-$MU**(3/2)) - $A4*(1-$MU**(4/2))', 
          'A1':0.7708, 'A2':0.0536, 'A3':-0.1577, 'A4':0.0481}, 
          doNotFit=['A1', 'A2', 'A3', 'A4'])
allfits['C2000 fixed'] = oi.bestfit.copy()
oi.show(logV=True, imFov=3, imPlx=0.815, showUV=False)

In [ ]:
# -- Teff=3500, logg=1.0, fit the 4 limb-darkening parameters (it does not converge)
oi.doFit({'diam':2.5, 
          'profile':'1 - $A1*(1-$MU**(1/2)) - $A2*(1-$MU**(2/2)) - $A3*(1-$MU**(3/2)) - $A4*(1-$MU**(4/2))', 
          'A1':0.7708, 'A2':0.0536, 'A3':-0.1577, 'A4':0.0481})
allfits['C2000 free'] = oi.bestfit.copy()
oi.show(logV=True, imFov=3, imPlx=0.815, showUV=False)
oi.showFit()

## Adding prior to help fit the LD parameters

To help fit the LD profile, we can add the constrain that $|a_k|<2$ for instance. This is done using the `prior` keyword in `doFit`: we pass a list of priors as tuples: `prior=[('np.abs(A1)', '<', 1), ...]` (no $ when you refer to parameters!). 

Now that we successfully fit more that 1 parameters, `PMOIRED` shows the correlation matrix bewteew parameters. Idealy, one wants parameters to be uncorrelated. Getting parameters highly correlated ($\gtrsim$95% in absolute value) means that the fit is probably unreliable. Sometimes, one can re-parametrise the model to decrease the correlations (see [tutorial #2](https://github.com/amerand/PMOIRED/blob/master/tutorials/Disk_tutorial_FS_CMa.ipynb)).

Question: How can you tell the fit is reliable?

In [ ]:
oi.setupFit({'obs':['V2'], 'max relative error':{'V2':0.5}})
m = {'diam':2.5, 
    'profile':'1 - $A1*(1-$MU**(1/2)) - $A2*(1-$MU**(2/2)) - $A3*(1-$MU**(3/2)) - $A4*(1-$MU**(4/2))', 
    'A1':0.7708, 'A2':0.0536, 'A3':-0.1577, 'A4':0.0481}
prior = [('np.abs(A1)', '<', 1), 
         ('np.abs(A2)', '<', 1), 
         ('np.abs(A3)', '<', 1), 
         ('np.abs(A4)', '<', 1)]
#prior = [('A1**2+A2**2+A3**2+A4**2', '<', 1.0)]
oi.doFit(m, prior=prior)
allfits['C2000 prior'] = oi.bestfit.copy()
oi.show(logV=True, imFov=3, imPlx=0.815, showUV=False)

## Limb darkening: power law

To fit the limb darkening, we need a simpler law with less parameters: we can use a power law as described in [Hestroffer (1997)](https://ui.adsabs.harvard.edu/abs/1997A%26A...327..199H/abstract): $I(\mu)/I(1) = \mu^\alpha$. 

Compare the result in terms of reduced $\chi^2$ and correlation between parameters

In [ ]:
oi.setupFit({'obs':['V2'],'max relative error':{'V2':0.5}})
prior=[('alpha', '>', 0)]
oi.doFit({'diam':2.5, 'profile':'$MU**$alpha', 'alpha':0.5}, prior=prior)

allfits['power law'] = oi.bestfit.copy()
oi.show(logV=True, imFov=3, imPlx=0.815, showUV=True)

<a id='resolved'></a>
# Oblate star and resolved flux

We can refine the model:
- make the stellar shape oblate using `incl` and `projang`: `incl` is the "inclination", which means that the stellar shape will be an ellipse with large axis will have diameter `diam` and small axis cos(`incl`)$\times$`diam`. The large axis orientation is set by `projang`: 0 for North and 90 for East. All angles are in degrees. 
- add a fully resolved component (e.i. Visibility==0 at every baseline): add a component with only a flux (or spectrum). Because we have 2 components, we have to differentiate the componenents by using `name,parameter` in the model. A fully resolved component `res` is define just as a flux `{'res,f':...}`. 
- Note that the star as an assumed total flux of 1, unless specified otherwise. We can express the flux of `star` as function of the flux of the resolved component so the total is 1.

What justifies, in the data, to add oblatness to the star and a fully resolved component?  

In [ ]:
oi.setupFit({'obs':['V2'],'max relative error':{'V2':0.5}})
prior=[('alpha', '>', 0)]
oi.doFit({'star,diam':2.5, 
          'star,profile':'$MU**$alpha', 
          'alpha':0.5, 
          'star,projang':45, 
          'star,incl':40, 
          'res,f':0.05, 
          'star,f':'1 - $res,f'
         },
           prior=prior)
# -- enforce that total flus is 1.0

allfits['oblate + resolved'] = oi.bestfit.copy()
oi.show(logV=True, imFov=3, imPlx=0.815, showUV=False)

# Adding a bright spot to the limb-darkened oblate model<a id='spot'></a>


Add a spot to the previous best fit model (as a uniform disk for example). The spot must be able to be at different position on the star: use `x` and `y` (in mas). Also you should give it a flux `f` (total flux, not surface brightness!). 

Fit the `T3PHI` and `V2` data. if the spot gets too small, you can use a prior to force its size to be a reasonable fraction of the stellar size (between ~1/2 and ~1/4 of the size of the star, considering we have data in the third lobe).

You may find that the fit does not converge depending of the initial parameters for the spot: it is because it is sensitive to the initial conditions, in particular the position of the spot. Try different initial positions on the star surface, as well as different flux for the spot

In [ ]:
# -- we also fit the closure phase
oi.setupFit({'obs':['T3PHI', 'V2'],
             'max relative error':{'V2':0.5}, 
             'max error':{'T3PHI':30}, 
                })
# -- taking the best model previously fitted
m = {'alpha':       0.521, # +/- 0.017
    'res,f':       0.0198, # +/- 0.0020
    'star,diam':   2.6349, # +/- 0.0079
    'star,incl':   20.94, # +/- 0.62
    'star,projang':89.64, # +/- 1.62
    'star,f':      '1 - $res,f',
    'star,profile':'$MU**$alpha',
    }
# -- adding a spot
m.update({'spot,diam':0.8, 'spot,x':1, 'spot,y':-1, 'spot,f':0.05})

# -- enforce that total flus is 1.0
m['star,f'] = '1 - $res,f - $spot,f'

prior = [('alpha', '>', 0),
         ('spot,diam', '<', 'star,diam/4'), 
         ('spot,diam', '>', 'star,diam/8'),
         ('spot,x**2+spot,y**2', '<', 'max(star,diam/2 - spot,diam/2, 0)**2')]

oi.doFit(m, prior=prior)
# -- using imMax to be able to see the stellar surface
oi.show(imFov=3, imPlx=0.815, logV=True, imMax='99', showUV=False)
oi.showFit()

## Randomized search to find the global best position for the spot<a id='grid_search'></a>

We saw the fit is sensitive to the initial position of the spot. 

We can use `oi.gridFit()` to explore various initial positions and run the corresponding fit. To define the exploration pattern, we use a dictionnary. see `?oi.gridFit` for more information, or the [notebook showing how to look for a companion around a star](https://github.com/amerand/PMOIRED/blob/master/examples/companion%20search%20AXCir.ipynb). In this case, a good choice is to use a 2D grid over the stellar surface. 

The pitch of the grid should be a fraction of the typical angular resolution, for example ~$\frac{\lambda}{3B_\mathrm{max}}$. The density of the grid will be checked *a posteriori* using the number of unique minima compared to the number of initial guesses.

You can use as options in `oi.gridFit` (check `?oi.gridFit` for full description): 
- `constrain=` to give a list of constrain on the initial parameters, for example for the spot to fall strictly on the stellar surface
- `prior=` which will be used while optimising the model.

`oi.showGrid` is used to show the result as a grid of the initial conditions and the position of the minima, with the final $\chi^2$ coloured coded. Note that initial positions as red crosses indicates the resulting fit was not satisfactory, e.g. if the uncertainty on the fitted parameter is large than the pitch of the grid. The gray lines indicates what initial positions led to a local minima. Ideally, one wants ~3 initial positions per minima to ensure that the space was samples correctly.

Compare you result to the ones presented in [Fig 2](https://www.aanda.org/articles/aa/full_html/2020/08/aa37832-20/F2.html) of the publication: we find the same position of the asymetry (midway to the South, slightly to the East).

In [ ]:

# -- we also fit the closure phase
oi.setupFit({'obs':['T3PHI', 'V2'],
                'max relative error':{'V2':0.5}, # ignore large uncertainties
                'max error':{'T3PHI':30}, # ignore large uncertainties
                })
# -- taking the best model previously fitted
m = {'alpha':       0.521, # +/- 0.017
    'res,f':       0.0198, # +/- 0.0020
    'star,diam':   2.6349, # +/- 0.0079
    'star,incl':   20.94, # +/- 0.62
    'star,projang':89.64, # +/- 1.62
    'star,f':      '1 - $res,f',
    'star,profile':'$MU**$alpha',
    }
# -- adding a bright spot
m.update({'spot,diam':0.8, 'spot,x':0.0, 'spot,y':0.0, 'spot,f':0.02})

# -- enforce that total flus is 1.0
m['star,f'] = '1 - $res,f - $spot,f'

# -- prior on the size of the spot
prior = [('alpha', '>', 0.01),
         ('spot,diam', '<', 'star,diam/4'), 
         ('spot,diam', '>', 'star,diam/8'),
         ('spot,x**2+spot,y**2', '<', 'max(star,diam/2 - spot,diam/2, 0)**2')]

# -- we define our exploration pattern (grid with step a fraction of angular resolution)
expl = {'grid':{'spot,x':(-m['star,diam']/2, m['star,diam']/2, 0.3), 
                'spot,y':(-m['star,diam']/2, m['star,diam']/2, 0.3)}}

# -- we constrain the exploration pattern so the spot is on the star
constrain = [('spot,x**2+spot,y**2', '<', '(star,diam/2)**2')]

# -- grid fit
oi.gridFit(expl, model=m, prior=prior, constrain=constrain)

allfits['oblate+resolved+bright spot'] = oi.bestfit.copy()

# -- show result of the grid:
oi.showGrid()

# -- show best fit model
oi.show(imFov=3, imMax='99', imPlx=0.815, logV=1, showUV=False)

# Comparing all the models we have fitted

In [ ]:
from IPython.core.display import HTML
data = {'model': list(allfits.keys()),
        'chi2': [round(allfits[k]['chi2'], 2) for k in allfits],
        'parameters': [pmoired.oimodels.dpfit.dispBest(allfits[k], asStr=True, color=False).replace('$', '\$').replace('\n', 'bBRr').replace('\'', '"') 
                       for k in allfits]
       }
    
df = pd.DataFrame(data)
html = df.to_html(justify='left')
html = html.replace('bBRr', '<br>')
display(HTML(html))

# Bonus: Use bootstrapping to evaluate the uncertainties<a id='bootstrapping'></a>

Use `oi.bootstrapFit(Nfits)` to perform `Nfits` fit with resampled data. `Nfits` should be of the order of the (number of baselines + number of triangle)x(number of files). In our case, the is 540. Although the bootstrapping is parallelised, running 540 fits will take several minutes on a typical laptop. You can use a smaller number, e.g. 100, to get an idea of the result. `oi.showBootstrap()` let you see the result as a corner plot.

Note that `oi.bootstrapFit()` will use your last best fit as initial parameters, as well as the fit's context and parameters (for example the `doNotFit`, etc.). 

In [ ]:
oi.bootstrapFit(100)
oi.showBootstrap()

# Bonus: fit a dark spot instead of a bright spot<a id='darkspot'></a>

A dark spot can be produce by giving a negative flux to the spot, which will be substracted frome the stellar surface. By default, `PMOIRED` does not allow components to be negative. To enable this, use `'ignore negative flux':True` in `oi.setupFit`. Then we initialise the spot with a negative flux, and add to the priors that its flux needs to remain negative. We have to specify in the prior an acceptable range: for example, if we tolerate the spot to be at most 1% of the total flux, we will give `('spot,f','<',0,0.01)`.

The best model finds a positive spot with +1% flux (our acceptable range): it means the data really favor a positive spot rather than a negative one.

In [ ]:
from importlib import reload
reload(pmoired.oimodels)
# -- we also fit the closure phase
oi.setupFit({'obs':['T3PHI', 'V2'],
                'max relative error':{'V2':0.5}, # ignore large uncertainties
                'max error':{'T3PHI':30}, # ignore large uncertainties
                'ignore negative flux':True,
            })

# -- taking the best model previously fitted
m = {'alpha':       0.521, # +/- 0.017
    'res,f':       0.0198, # +/- 0.0020
    'star,diam':   2.6349, # +/- 0.0079
    'star,incl':   20.94, # +/- 0.62
    'star,projang':89.64, # +/- 1.62
    'star,f':      '1 - $res,f',
    'star,profile':'$MU**$alpha',
    }
# -- adding a bright spot
m.update({'spot,diam':0.8, 'spot,x':0.0, 'spot,y':0.0, 'spot,f':-0.02})

# -- enforce that total flus is 1.0
m['star,f'] = '1 - $res,f - $spot,f'

# -- prior on the size of the spot
prior = [('alpha', '>', 0.01),
         ('spot,diam', '<', 'star,diam/4'), 
         ('spot,diam', '>', 'star,diam/8'),
         ('spot,f', '<', 0, 1e-2),
         ('spot,x**2+spot,y**2', '<', 'max(star,diam/2 - spot,diam/2, 0)**2')]

# -- we define our exploration pattern (grid with step a fraction of angular resolution)
expl = {'grid':{'spot,x':(-m['star,diam']/2, m['star,diam']/2, 0.3), 
                'spot,y':(-m['star,diam']/2, m['star,diam']/2, 0.3)}}

# -- we constrain the exploration pattern so the spot is on the star
constrain = [('spot,x**2+spot,y**2', '<', '(star,diam/2)**2')]

# -- grid fit
oi.gridFit(expl, model=m, prior=prior, constrain=constrain)

allfits['oblate+resolved+dark spot'] = oi.bestfit.copy()

# -- show result of the grid:
oi.showGrid()

# -- show best fit model
oi.show(imFov=3, imMax='99', imPlx=0.815, logV=1, showUV=False)